In [1]:
import numpy as np
np.random.seed(0)
import mltools as ml
import matplotlib.pyplot as plt # use matplotlib for plotting with inline plot
%matplotlib inline

In [2]:
X = np.genfromtxt("C:\\Users\\regin\\Desktop\\CS 178\\Project\\X_train.txt", delimiter = None)
Y = np.genfromtxt("C:\\Users\\regin\\Desktop\\CS 178\\Project\\Y_train.txt", delimiter = None)

Xte = np.genfromtxt("C:\\Users\\regin\\Desktop\\CS 178\\Project\\X_test.txt", delimiter = None)

In [4]:
# Xtr, Xva, Ytr, Yva = ml.splitData(X, Y) # Default is 80% train & 20% val
# Xtr, Ytr = ml.shuffleData(Xtr, Ytr)

Xtr, Ytr = ml.shuffleData(X,Y)

In [6]:
# Scaling the data to avoid numerical data issues (esp for when features are large)
XtrP, params = ml.rescale(Xtr)
XteP, _ = ml.rescale(Xte, params)
#XvaP, _ = ml.rescale(Xva, params)

In [7]:
XtrP = ml.transforms.fpoly(Xtr, 2, bias = False)
#XteP = ml.transforms.fpoly(XteP, 2, bias = False)
#XvaP = ml.transforms.fpoly(Xva, 2, bias = False)

XtrP, params = ml.rescale(XtrP)
#XteP, _ = ml.rescale(XteP, params)
#XvaP, _ = ml.rescale(XvaP, params)

In [27]:
XtrLF = ml.transforms.fsubset(XtrP, 9)
XtrLF = np.atleast_2d(XtrLF)
XtrLF = XtrLF[0][0]
XtrLF, params = ml.rescale(XtrLF)
#print(XtrLF)

In [28]:
XtrLF = ml.transforms.fpoly(XtrLF, 4, bias = False)
#XteP = ml.transforms.fpoly(XteP, i, bias = False)
#XvaP = ml.transforms.fpoly(Xva, 3, bias = False)

XtrLF, params = ml.rescale(XtrLF)
#XteP, _ = ml.rescale(XteP, params)
#XvaP, _ = ml.rescale(XvaP, params)

In [26]:
# Train model

learner = ml.linearC.linearClassify()
learner.train(XtrP, Ytr, initStep = 0.1, stopTol = 1e-20, stopIter = 2000)
    
        # When first debugging, can keep stopTol large & stopIter
        # but may want to tune for Kaggle
#probs = learner.predictSoft(XteP)

KeyboardInterrupt: 

In [ ]:
nFolds = 10

Jt = np.zeros(nFolds)
Jv = np.zeros(nFolds)

for i in range(nFolds):
    Xti, Xvi, Yti, Yvi = ml.crossValidate(XtrLF, Ytr, nFolds, i)
    
    learner = ml.linearC.linearClassify()
    learner.train(Xti, Yti, initStep = 0.01, stopTol = 1e-20, stopIter = 500)
    
    Jt[i] = learner.auc(Xti, Yti)
    Jv[i] = learner.auc(Xvi, Yvi)

print("{0:>15}: {1:.4f}".format('Train AUC', np.mean(Jt)))
print("{0:>15}: {1:.4f}".format('Validation AUC', np.mean(Jv)))

In [17]:
XteP = ml.transforms.fpoly(XteP, 3, bias = False)
XteP, _ = ml.rescale(XteP, params)
probs = learner.predictSoft(XteP)

In [18]:
Y_sub = np.vstack([np.arange(Xte.shape[0]), probs[:,1]]).T
np.savetxt("C:\\Users\\regin\\Desktop\\CS 178\\Project\\Y_testLC2.txt", Y_sub, '%d, %.5f', header = 'ID,Prob1', comments = '', delimiter = ',')

2nd order w/ rescale & w/o reg
initstep 0.1, stopTol = 1e-10, stopIter = 500
      Train AUC: 0.6702
 Validation AUC: 0.6635
- ~~~~~~~ definite improvement compared to when usisng 100 iterations, Train AUC: 0.5636 Validation AUC: 0.5546~~~~~~ -

initstep 0.1, stopTol = 1e-15, stopIter = 1000
      Train AUC: 0.6746
 Validation AUC: 0.6670
 
initstep 0.1, stopTol = 1e-15, stopIter = 2000

      Train AUC: 0.6976
 Validation AUC: 0.6691
 
 XtrP, Yt, initStep = 0.01, stopTol = 1e-20, stopIter = 2000
       Train AUC: 0.6896
 Validation AUC: 0.6680
 
XtrP, Yt, initStep = 0.05, stopTol = 1e-20, stopIter = 2000
  
      Train AUC: 0.6969
 Validation AUC: 0.6676
 
Full data set (no CV) TestLC1
      Train AUC: 0.6777
 Validation AUC: 0.6815
 
 3rd degree 10 features
 initStep = 0.1, stopTol = 1e-20, stopIter = 100
       Train AUC: 0.6046
 Validation AUC: 0.6041
 
initStep = 0.1, stopTol = 1e-20, stopIter = 500
      Train AUC: 0.6193
 Validation AUC: 0.6179
 
 4th degree nine features
initStep = 0.1, stopTol = 1e-20, stopIter = 500

1st order
initStep = 0.5, stopTol = 1e-6, stopIter = 100
      Train AUC: 0.6639
 Validation AUC: 0.6543
 
 
2nd order w/o rescale
      Train AUC: 0.5006
 Validation AUC: 0.5040


2nd order w/ rescale (but no reg)
      Train AUC: 0.5636
 Validation AUC: 0.5546
 
reg = 0.1/0.2
      Train AUC: 0.4995
 Validation AUC: 0.5040
 
 reg = 0.3 
      Train AUC: 0.4625
 Validation AUC: 0.4595

tried different reg; only best value is 0 soooo am not too sure

In [21]:
XtrLF = ml.transforms.fpoly(XtrLF, 3, bias = False)
#XteP = ml.transforms.fpoly(XteP, i, bias = False)
XvaP = ml.transforms.fpoly(Xva, 3, bias = False)

XtrLF, params = ml.rescale(XtrLF)
#XteP, _ = ml.rescale(XteP, params)
XvaP, _ = ml.rescale(XvaP, params)

NameError: name 'Xva' is not defined

## 3rd degree